Now that we've covered the basic components of the Pipeline API, let's construct a pipeline that we might want to use in an algorithm. 

To start, let's first create a filter to narrow down the types of securities coming out of our pipeline. In this example, we will create a filter to select for securities that meet the following criteria:

    Is a primary share
    Is listed as a common stock
    Is not a depositary receipt (ADR/GDR)
    Is not trading over-the-counter (OTC)
    Is not when-issued (WI)
    Doesn't have a name indicating it's a limited partnership (LP)
    Is not an ETF (has Morningstar fundamental data)

Why do we have such a criteria?

When it comes to forming a trading universe its best to select Primary common stocks. The stocks that are (ADR, OTC, WI, LP) have low liquidity and are not available to trade at every brokerage. We also exclude ETF when it comes to ranking stock say for example on P/E ratios, it does not make sense to include all the ETF.

Having said that lets build our universe.

In [1]:
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.filters.fundamentals import IsPrimaryShare

from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.data import USEquityPricing

Primary Shares

In [2]:
primary_share = IsPrimaryShare()

Common Stock

In [3]:
#'ST00000001' indicates common stock.
common_stock = Fundamentals.security_type.latest.eq('ST00000001')

Not a Depositary receipt

In [4]:
not_depositary = ~Fundamentals.is_depositary_receipt.latest

Not an OTC

In [5]:
not_otc = ~Fundamentals.exchange_id.latest.startswith('OTC')

Not WI or 'When-issued'

In [6]:
not_wi = ~Fundamentals.symbol.latest.endswith('.WI')

Not a LP

In [7]:
# Use a regex to exclude LP
not_lp_name = ~Fundamentals.standard_name.latest.matches('.* L[. ]?P.?$')

Have a market cap that is Not-Null

In [8]:
have_market_cap = Fundamentals.market_cap.latest.notnull()

Build our Universe

In [9]:
tradable_stocks = (

    primary_share
    & common_stock
    & not_depositary
    & not_otc
    & not_wi
    & not_lp_name
    & have_market_cap
)

Lets build our pipeline. As before we will use the 10 and 30 day moving average and get the percent difference between the two.

In [10]:
def make_pipeline():
    
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=tradable_stocks
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=tradable_stocks
    )

    # Percent difference factor.
    percent_difference = (mean_10 - mean_30) / mean_30
    
    return Pipeline(
    
        columns = {
            
            'percent_diff':percent_difference
            
        },screen = tradable_stocks
    )
    
    

In [11]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

In [12]:
output.head()

percent_diff
2016-01-11 00:00:00+00:00 Equity(2 [ARNC])       0.012563
                          Equity(21 [AAME])     -0.002626
                          Equity(24 [AAPL])     -0.060071
                          Equity(31 [ABAX])      0.021732
                          Equity(41 [ARCB])     -0.043046

In [13]:
len(output)

4311

Suppose we think that this percent_diff factor is indicate of future returns. We decide to go long the top 75 stocks and go short the bottom 75 stocks.

In [14]:
def make_pipeline():
    
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=tradable_stocks
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=tradable_stocks
    )

    # Percent difference factor.
    percent_difference = (mean_10 - mean_30) / mean_30
    
    longs = percent_difference.top(75)
    
    shorts = percent_difference.bottom(75)
    
    securities_to_trade = (longs | shorts)
    
    return Pipeline(
    
        columns = {
            
            'percent_diff':percent_difference
            
        },screen = securities_to_trade
    )
    
    

In [15]:
output = run_pipeline(make_pipeline(),
                     '2016-1-11',
                     '2016-1-11')

output.head()

percent_diff
2016-01-11 00:00:00+00:00 Equity(112 [ACY])        0.106386
                          Equity(393 [AMSC])       0.218685
                          Equity(624 [ATW])       -0.199640
                          Equity(1587 [CDZI])      0.158030
                          Equity(1923 [CSPI])      0.107657

In [16]:
len(output)

150

Sure enough we have 150 stocks (longs and shorts) to trade for January 11,2016.

We can use the above pipeline to select securities to backtest this (random) strategy.